
This notebook contains examples of 4 different types of pipelines:
* Text to Embedding
* Text to Text translation
* Speech to Embedding
* Speech to Text translation

### Setup common config

In [3]:
import torch
device = torch.device("cpu")

### Loading Models (speech encoder + text decoder + tokenizer)

In [2]:
from sonar.models.sonar_speech.loader import load_sonar_speech_model
from sonar.models.sonar_text import (
    load_sonar_text_decoder_model,
    load_sonar_text_encoder_model,
    load_sonar_tokenizer,
)

In [4]:
speech_encoder_model = load_sonar_speech_model("sonar_speech_encoder_eng", device=device).eval()

In [5]:
text_encoder_model = load_sonar_text_encoder_model("text_sonar_basic_encoder", device=device).eval()

In [6]:
text_decoder_model = load_sonar_text_decoder_model("text_sonar_basic_decoder", device=device).eval()

In [7]:
# tokenizer is compatible with nllb tokenizer logic already
text_tokenizer = load_sonar_tokenizer("text_sonar_basic_encoder")

### Running Speech Data Pipelines

In [8]:
from sonar.inference_pipelines.speech import SpeechToEmbeddingModelPipeline, SpeechToTextModelPipeline

In [10]:
s2vec_model = SpeechToEmbeddingModelPipeline(encoder=speech_encoder_model)
s2vec_model.predict(["../tests/integration_tests/data/audio_files/audio_1.wav",
                     "../tests/integration_tests/data/audio_files/audio_2.wav"])

tensor([[ 3.6588e-03,  1.7128e-03,  9.0594e-03,  ...,  5.7871e-05,
          4.5227e-03, -2.3104e-03],
        [ 7.0923e-04,  6.1351e-03, -2.6858e-03,  ...,  2.6961e-03,
         -6.0307e-03,  1.7454e-03]])

In [12]:
s2t_model = SpeechToTextModelPipeline(encoder=speech_encoder_model,
                                      decoder=text_decoder_model,
                                      tokenizer=text_tokenizer)

s2t_model.predict(["../tests/integration_tests/data/audio_files/audio_1.wav",
                   "../tests/integration_tests/data/audio_files/audio_2.wav"], target_lang="eng_Latn")

['Television reports show white smoke coming from the plant.',
 'These couples may choose to make an adoption plan for their baby.']

### Running Text Data Pipeline

In [13]:
from sonar.inference_pipelines.text import TextToTextModelPipeline, TextToEmbeddingModelPipeline

In [14]:
text_embedding_pipeline = TextToEmbeddingModelPipeline(text_encoder_model, text_tokenizer)
text_to_text_pipeline = TextToTextModelPipeline(text_encoder_model, text_decoder_model, text_tokenizer)

#### Applying models

In [16]:
data_source = "./eng_flores200_dev_sample.tsv"
text_emb = text_embedding_pipeline.predict(data_source, source_lang="eng_Latn")
text_emb

tensor([[ 1.3900e-02,  7.9963e-03, -1.3692e-02,  ..., -4.1727e-03,
          8.6479e-03,  1.1009e-03],
        [ 2.6909e-03,  1.3037e-02, -4.8724e-03,  ..., -7.1355e-03,
          2.3920e-04,  8.0676e-03],
        [-6.1483e-03, -1.0119e-03,  5.2443e-03,  ..., -8.6812e-03,
         -5.3998e-03,  3.4272e-03],
        ...,
        [-3.0806e-03, -4.0161e-03,  8.5773e-03,  ..., -5.5559e-03,
         -3.8027e-03, -9.9625e-03],
        [ 2.0388e-02,  1.9085e-04, -5.5832e-05,  ..., -8.3714e-04,
         -4.2589e-03, -4.5399e-03],
        [-1.3931e-03,  1.5254e-03,  5.5113e-03,  ..., -5.0019e-03,
          1.3234e-02,  3.0003e-03]])

In [17]:
text_translation = text_to_text_pipeline.predict(data_source, source_lang="eng_Latn", target_lang="fra_Latn")
text_translation

['Nombre',
 'Nous avons maintenant des souris âgées de 4 mois qui ne sont pas diabétiques et qui étaient autrefois diabétiques, a-t-il ajouté.',
 "Le docteur Ehud Ur, professeur de médecine à l'université Dalhousie à Halifax, en Nouvelle-Écosse, et président de la division clinique et scientifique de l'Association canadienne du diabète, a averti que la recherche était encore à ses débuts.",
 "À l'instar d'autres spécialistes, il est sceptique quant à savoir si le diabète peut être guéri, notant que ces découvertes n'ont pas d'importance pour les personnes déjà atteintes de diabète de type 1.",
 "Lundi, Sara Danius, secrétaire permanent du Comité Nobel de littérature à l'Académie suédoise, a annoncé publiquement, lors d'une émission radiophonique à l'émission Sveriges Radio en Suède, que l'équipe incapable de contacter Bob Dylan pour obtenir le prix Nobel de littérature 2016, avait abandonné ses efforts pour le recevoir.",
 '"Danius a dit:" Pour l\'instant, nous ne faisons rien. J\'ai a